In [39]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

2023-06-16:01:46:36,555 INFO     [workspace.py:291] Found the config file in: /config.json


Workspace name: quick-starts-ws-235836
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-235836


In [40]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "project1"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
try:
    target = ComputeTarget(workspace=ws, name=cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    target = ComputeTarget(ws, cluster_name, compute_config)
target.wait_for_completion(show_output=True)


Jobrunning
Running


In [41]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : uniform(0.05,0.1),
        '--max_iter' : choice(0,40,80,120,140)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory= '.', script ='train.py', compute_target = target, environment =sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src, hyperparameter_sampling=ps, primary_metric_name='accuracy',
                                     primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, max_total_runs=15, max_concurrent_runs=4)

In [42]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyper_drive_run = exp.submit(hyperdrive_config)
RunDetails(hyper_drive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [45]:
import joblib
# Get your best run and save the model from that run.

best_run = hyper_drive_run.get_best_run_by_primary_metric()
joblib.dump(best_run, 'best_run_hyper_drive.joblib')

['best_run_hyper_drive.joblib']

In [47]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")


KeyboardInterrupt: 

In [ ]:
from train import clean_data
import pandas as pd
# Use the clean_data function to clean your data.
x, y = clean_data(ds)
dataset = pd.concat([x,y], axis=1)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=3)

In [ ]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output = True)

# Retrieve and save your best automl model.
# Retrieve and save your best automl model.
best_run_automl = remote_run.get_best_child()
print(best_run_automl)
best_run_automl.save_model('./')

In [ ]:
# Retrieve and save your best automl model.
best_run_automl = remote_run.get_output()
print(best_run_automl)
best_run_automl.save_model('./')

In [38]:
#Cluster Clean up
ComputeTarget.delete

<function azureml.core.compute.compute.ComputeTarget.delete(self)>